In [4]:
# from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.base import clone

# Обучение модели

## Загружаем данные из pickle

In [7]:
model_file = open('processed_data2.pkl', 'rb')
data = pickle.load(model_file)
model_file.close()
data

,duration,rounds,f1_age,f2_age,f1_isHomeCity,f1_isHomeCountry,f1_isHomeTimezone,f2_isHomeCity,f2_isHomeCountry,f2_isHomeTimezone,...,grappling_accuracy_difference,hitsTotal_difference,takedownTotal_difference,submissionAttempts_difference,takeovers_difference,hitsBodyTotal_difference,hitsLegsTotal_difference,weight_difference,height_difference,eventDate
id,,,,,,,,,,,,,,,,,,,,,
5201,104.0,1.0,27.0,34.0,0,0,0,0,0,0,...,0.0,4.0,3.0,1.0,0.0,0.0,1.0,-18.6,-10.2,1993-11-12 00:00:00
5202,52.0,1.0,24.0,23.0,0,1,0,0,1,0,...,0.0,2.0,2.0,1.0,0.0,0.0,0.0,2.3,-7.6,1993-11-12 00:00:00
5203,59.0,1.0,23.0,34.0,0,1,0,0,0,0,...,0.0,-14.0,0.0,0.0,0.0,0.0,-2.0,26.8,-2.5,1993-11-12 00:00:00
5204,57.0,1.0,29.0,27.0,0,1,0,0,0,0,...,0.0,-12.0,0.0,-1.0,-1.0,0.0,0.0,13.6,0.0,1993-11-12 00:00:00
5205,138.0,1.0,27.0,30.0,0,0,0,0,1,0,...,0.0,7.0,1.0,0.0,0.0,0.0,2.0,-9.5,0.0,1993-11-12 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5310,260.0,1.0,25.0,25.0,0,1,0,0,1,1,...,-1.0,19.0,1.0,1.0,0.0,0.0,0.0,0.0,7.6,2019-12-08 05:00:00
6338,300.0,3.0,30.0,22.0,0,1,0,0,0,0,...,0.0,71.0,1.0,0.0,0.0,1.0,-2.0,0.0,-5.1,2019-12-08 05:00:00
5311,300.0,5.0,31.0,35.0,0,0,0,0,0,0,...,0.0,6.0,2.0,-1.0,0.0,2.0,1.0,0.0,-2.5,2019-12-14 00:00:00


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6285 entries, 5201 to 6340
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   duration                        6285 non-null   float64       
 1   rounds                          6285 non-null   float64       
 2   f1_age                          6285 non-null   float64       
 3   f2_age                          6285 non-null   float64       
 4   f1_isHomeCity                   6285 non-null   int64         
 5   f1_isHomeCountry                6285 non-null   int64         
 6   f1_isHomeTimezone               6285 non-null   int64         
 7   f2_isHomeCity                   6285 non-null   int64         
 8   f2_isHomeCountry                6285 non-null   int64         
 9   f2_isHomeTimezone               6285 non-null   int64         
 10  winner                          6285 non-null   bool          
 11  K

## Выбираем независимые и целевую переменные

### Выбираем период для обучающей и тестовой выборки 

In [10]:
def X_y_split(df , feature_names, target_column, basline_column):
    """
  Функция разделяет выборку на трейн , тест и выделяет бейслайн
  На входе pandas dataframe и перечень фичей для обучения/ целевая переменная/ бейслайн фича,
  На выходе train/test X, Y, baseline dataframes
  """
    X = df[feature_names]
    Y= df[target_column]
    Y_baseline = df[basline_column]
    X_train, X_test, Y_train, Y_test = X.iloc[0:-100,:], X.iloc[-100:], Y.iloc[0:-100], Y.iloc[-100:] #В качестве тестового множества берем последние 100 дней
    Y_train_baseline , Y_test_baseline = Y_baseline[0:-100],  Y_baseline[-100:]
    return X, Y, Y_baseline, X_train, X_test, Y_train, Y_test,  Y_train_baseline , Y_test_baseline

In [38]:
X = data.drop(columns=['winner'])
y = data[['winner', 'eventDate']]
X_train = X[X['eventDate'].dt.year < 2019].drop(columns=['eventDate'])
X_test = X[X['eventDate'].dt.year >= 2019].drop(columns=['eventDate'])
y_with_date_to_train = y[y['eventDate'].dt.year < 2019]
y_with_date_to_test = y[y['eventDate'].dt.year >= 2019]

In [39]:
y_train = y_with_date_to_train['winner']
y_test = y_with_date_to_test['winner']

In [40]:
y_test.shape

(439,)

## Пробуем XGboost classifier

In [18]:
import xgboost as xgb

### Преобразовываем независимые и целевую переменные

In [41]:
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

### Настраиваем дефолтные параметры

In [42]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations

### Обучаем модель XGBoost Classifier

In [43]:
xgb_model = xgb.train(param, D_train, steps)

[18:04:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


### Предсказываем целевую переменную

In [44]:
preds = xgb_model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

Accuracy = 0.7129840546697038


In [47]:
scaler = StandardScaler()
cover_X_train = scaler.fit_transform(X_train)
cover_X_test = scaler.transform(X_test)

In [48]:
cover_X_train.shape

(5846, 24)

In [49]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [284]:

model= models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(24,)))
# model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [285]:
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [286]:
model.fit(cover_X_train, y_train, epochs=3, batch_size=32)

Epoch 1/3
183/183 [==============================] - 1s 3ms/step - loss: 0.4723 - accuracy: 0.7716
Epoch 2/3
183/183 [==============================] - 1s 3ms/step - loss: 0.4113 - accuracy: 0.8038
Epoch 3/3
183/183 [==============================] - 1s 3ms/step - loss: 0.3866 - accuracy: 0.8165


In [287]:
model.evaluate(cover_X_test, y_test)

14/14 [==============================] - 0s 2ms/step - loss: 0.5284 - accuracy: 0.7517


[0.5284481644630432, 0.751708447933197]

In [288]:
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [302]:
perceptron = Perceptron(max_iter = 1000, tol = 0.001)
perceptron.fit(cover_X_train, y_train)

Perceptron()

In [303]:
perceptron.score(cover_X_test, y_test)

0.6378132118451025

In [294]:
sgd = SGDClassifier(max_iter = 1000, tol = 0.001)
sgd.fit(cover_X_train, y_train)
y_pred = sgd.predict(cover_X_test)

In [296]:
accuracy_score(y_test,y_pred)

0.6924829157175398

In [297]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(cover_X_train, y_train)
y_pred = decision_tree.predict(cover_X_test)

In [298]:
accuracy_score(y_test,y_pred)

0.7175398633257403

In [299]:
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
gaussian.fit(cover_X_train, y_train)
y_pred = gaussian.predict(cover_X_test)

In [300]:
accuracy_score(y_test,y_pred)

0.6697038724373576